# Extraction of Relevant Action and Object Knowledge from the Web

Before starting this tutorial, make sure the necessary packages are installed (see requirements.txt). Additionally, the following files need to be downloaded / extracted into this folder:

- *foodon.owl* - FoodOn used for extracting information about fruits and vegetables (found [here](https://github.com/FoodOntology/foodon))
- *numberbatch-en.txt* - ConceptNet Numberbatch embeddings for object property extraction (download *English-only V. 19.08* from [here](https://github.com/commonsense/conceptnet-numberbatch?tab=readme-ov-file#downloads))
- *NASARI_embed_english.txt* - NASARI embeddings also used for object property extraction (download *English - Embed(Wiki)* from [here](http://lcl.uniroma1.it/nasari/#two))

## Extraction of Action Knowledge

The extraction of knowledge about different and relevant actions consists of three main steps:

1. Setting the central verb & providing an exemplary sentence (e.g. 'cut')
2. Extracting synonyms and hyponyms from WordNet & VerbNet
3. Filtering the extracted words on their relevance using a recipe and a WikiHow corpus

In [ ]:
# imports
import pandas as pd

# download wordnet & verbnet corpus
import nltk
nltk.download('wordnet')
nltk.download('verbnet')
from nltk.corpus import verbnet, wordnet

### Extracting Synonyms and Hyponyms from WordNet & VerbNet

In [ ]:
# setting the target action and an exemplary sentence
target_action = "cut"
verbs = []

# iterating over all WordNet synsets containing the verb and ...
synsets = wordnet.synsets(target_action, pos=wordnet.VERB)
print(f"{len(synsets)} synsets found for '{target_action}'")
for syn in synsets:
    # ... gathering all synonyms & direct hyponyms
    verbs.extend(syn.lemma_names())
    for h in syn.hyponyms():
        verbs.extend(h.lemma_names())

    # ... getting the associated VerbNet class
    key = str(syn.lemmas()[0].key()).replace("::", "")
    vn_classes = verbnet.classids(wordnetid=key)
    for vn_class in vn_classes:
        verbs.extend(verbnet.lemmas(vn_class))

# removing duplicates and printing results
verbs = set(verbs)
print(f"{len(verbs)} synonyms or hyponyms found for '{target_action}'")

In [ ]:
# pre-process the found synonyms and hyponyms
filtered_verbs = {v.split('_')[0] for v in verbs}
filtered_verbs = sorted(set(filtered_verbs))

print(f"{len(filtered_verbs)} remaining words:")
for verb in filtered_verbs:
    print(verb)

### Filtering the extracted verbs

In [ ]:
# read the (extracted) occurrence data
v_occurrences = "./verb_occurrences.csv"
voc_dat = pd.read_csv(v_occurrences)

# remove all verbs with 0 occurrences
most_used = voc_dat[(voc_dat['SUM'] > 0)]
print(f"{len(most_used)} verbs that occur at least once")

# remove all verbs with too few available sentences (Step Desc <= threshold)
thresh = 100
most_used = most_used[(most_used['Step Desc'] >= thresh)]
print(f"{len(most_used)} verbs that occur in more than {thresh} step descriptions:")
print(most_used['Verb'].to_string(index=False))

## Extraction of Object Knowledge

The extraction of knowledge about different objects and their task-specific properties consists of four main steps:

1. Choosing a group of relevant objects (e.g. 'Fruits & Vegetables')
2. Gathering all available objects of that group from a suitable taxonomy or ontology (e.g. FoodOn)
3. Filtering the extracted objects using fitting text corpora (e.g. Recipe1M+)
4. Gathering information about task-specific properties using 3 different word embeddings (GloVe, NASARI & ConceptNet Numberbatch) and a recipe corpus

In [1]:
# imports
from rdflib import Graph, Literal, Namespace, RDF, RDFS, URIRef
from rdflib.plugins.sparql import prepareQuery
import pandas as pd
import gensim
import torch
import torchtext

### Extracting fruits and vegetables from the FoodOn

In [6]:
# load the ontology and set the namespace prefixes
foodon_loc = "./foodon.owl"
g = Graph()
g.parse(foodon_loc)

FOOD = Namespace("http://purl.obolibrary.org/obo/")
RDFS = Namespace("http://www.w3.org/2000/01/rdf-schema#")

# get the fruit data through the SPARQL query 
query = prepareQuery(   
    """
    SELECT ?fruit_label (SAMPLE(?fruit_id) AS ?rndm_fruit_id) (SAMPLE(?def) AS ?rndm_def)
    WHERE {
        ?fruit_id rdfs:label ?label.
        ?fruit_id rdfs:subClassOf+ food:PO_0009001.
        OPTIONAL { ?fruit_id food:IAO_0000115 ?def. }

        BIND (LCASE(STR(?label)) AS ?str_label).
        BIND (STRBEFORE(?str_label, "(") AS ?fruit_label).
        FILTER CONTAINS(?str_label, "whole").
        FILTER NOT EXISTS { ?fruit_id rdfs:subClassOf* food:PO_0030104. }
        FILTER (?fruit_id != food:FOODON_03304644).
    }
    GROUP BY ?fruit_label
    ORDER BY ?fruit_label
    """,
    initNs={"food": FOOD, "rdfs": RDFS}
)
fruits = g.query(query)

# get the vegetable data through the SPARQL query 
query = prepareQuery(
    """
    SELECT ?veg_label (SAMPLE(?veg_id) AS ?rndm_veg_id) (SAMPLE(?def) AS ?rndm_def)
    WHERE {
        ?veg_id rdfs:label ?label.
        ?veg_id rdfs:subClassOf+ food:FOODON_03302008.
        OPTIONAL { ?veg_id food:IAO_0000115 ?def. }

        BIND (LCASE(STR(?label)) AS ?str_label).
        BIND (STRBEFORE(?str_label, "(") AS ?veg_label).
        FILTER NOT EXISTS { ?veg_id rdfs:subClassOf* food:FOODON_03302007. }
    }
    GROUP BY ?veg_label
    ORDER BY ?veg_label
    """,
    initNs={"food": FOOD, "rdfs": RDFS}
)
veggies = g.query(query)

# convert query results into panda dataframes for further analysis
fruit_list = [(str(row[0]), str(row[1]), str(row[2])) for row in fruits]
veggie_list = [(str(row[0]), str(row[1]), str(row[2])) for row in veggies]

fruit_df = pd.DataFrame(fruit_list, columns=["label", "rndm_id", "rndm_def"])
veggie_df = pd.DataFrame(veggie_list, columns=["label", "rndm_id", "rndm_def"])
combined_df = pd.concat([fruit_df, veggie_df], ignore_index=True)
print(f'Fruits: {len(fruit_df)}\nVegetables: {len(veggie_df)}\nCombined: {len(combined_df)}')
print(combined_df)

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#dateTime, Converter=<function parse_datetime at 0x7f6bc85698b0>
Traceback (most recent call last):
  File "/home/jan-philipp/.local/lib/python3.8/site-packages/isodate/isodatetime.py", line 51, in parse_datetime
    datestring, timestring = datetimestring.split('T')
ValueError: not enough values to unpack (expected 2, got 1)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/jan-philipp/.local/lib/python3.8/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "/home/jan-philipp/.local/lib/python3.8/site-packages/isodate/isodatetime.py", line 53, in parse_datetime
    raise ISO8601Error("ISO 8601 time designator 'T' missing. Unable to"
isodate.isoerror.ISO8601Error: ISO 8601 time designator 'T' missing. Unable to parse datetime string '2022-03-22 15:59:09'

Fruits: 257
Vegetables: 31
Combined: 288
                   label                                         rndm_id  \
0               acerola   http://purl.obolibrary.org/obo/FOODON_00003593   
1                 acorn   http://purl.obolibrary.org/obo/FOODON_00003591   
2          acorn squash   http://purl.obolibrary.org/obo/FOODON_00003599   
3                  akee   http://purl.obolibrary.org/obo/FOODON_00003489   
4      alaska blueberry   http://purl.obolibrary.org/obo/FOODON_00004209   
..                   ...                                             ...   
283        salad greens   http://purl.obolibrary.org/obo/FOODON_03310789   
284  savoy cabbage head   http://purl.obolibrary.org/obo/FOODON_00003031   
285            scallion   http://purl.obolibrary.org/obo/FOODON_00003697   
286  swamp cabbage head   http://purl.obolibrary.org/obo/FOODON_03000060   
287         winged bean   http://purl.obolibrary.org/obo/FOODON_00002810   

                                              

### Filter the fruits and vegetables using WikiHow and Recipe1M+ data

In [5]:
f_occurrences = "./fruit_occurrences.csv"

# read and map the (extracted) occurrence data
foc_dat = pd.read_csv(f_occurrences)
foc_dat = foc_dat.astype({'Recipes-Title':'int','Recipes-Title [%]':'float', 'Recipes-Steps':'int','Recipes-Steps [%]':'float',
                'WikiHow-Title':'int','WikiHow-Title [%]':'float', 'WikiHow-TitleDescription':'int','WikiHow-TitleDescription [%]':'float',
                'WikiHow-Method':'int','WikiHow-Method [%]':'float', 'WikiHow-StepHeadline':'int','WikiHow-StepHeadline [%]':'float',
                'WikiHow-StepDescription':'int','WikiHow-StepDescription [%]':'float'})

# remove all items with too few occurrences in any column (less than 1%)
thresh = 0.01
most_used = foc_dat[(foc_dat['Recipes-Title [%]'] >= thresh) | (foc_dat['Recipes-Steps [%]'] >= thresh) | (foc_dat['WikiHow-Title [%]'] >= thresh) | 
                  (foc_dat['WikiHow-TitleDescription [%]'] >= thresh) | (foc_dat['WikiHow-Method [%]'] >= thresh) | (foc_dat['WikiHow-StepHeadline [%]'] >= thresh) |
                  (foc_dat['WikiHow-StepDescription [%]'] >= thresh)]
print(most_used)

          Name                                             IRI       Type  \
0       pepper  http://purl.obolibrary.org/obo/FOODON_00003520      Fruit   
1        olive  http://purl.obolibrary.org/obo/FOODON_03317509      Fruit   
2        lemon  http://purl.obolibrary.org/obo/FOODON_03301441      Fruit   
3       tomato  http://purl.obolibrary.org/obo/FOODON_03309927      Fruit   
4       orange  http://purl.obolibrary.org/obo/FOODON_03309832      Fruit   
5         lime  http://purl.obolibrary.org/obo/FOODON_00003661      Fruit   
6        apple  http://purl.obolibrary.org/obo/FOODON_03301710      Fruit   
7         bean  http://purl.obolibrary.org/obo/FOODON_03301403  Vegetable   
8      coconut  http://purl.obolibrary.org/obo/FOODON_00003449      Fruit   
9       banana  http://purl.obolibrary.org/obo/FOODON_00004183      Fruit   
10      almond  http://purl.obolibrary.org/obo/FOODON_00003523      Fruit   
11   pineapple  http://purl.obolibrary.org/obo/FOODON_00003459      Fruit   

### Extract object properties using 3 different embeddings

In [ ]:
# prepare the list of possible fruits and possible food parts
parts = ['core', 'shell', 'peel', 'stem']
foods = list(most_used['Name'])

In [ ]:
# GloVe embeddings
glove_sim = 0.5
glove = torchtext.vocab.GloVe(name="6B", dim=50)

for f in foods:
    for p in parts:
        sim = torch.cosine_similarity(glove[f].unsqueeze(0), glove[p].unsqueeze(0)).item()
        if sim >= glove_sim:
            print(f'[GloVe] {f} hasPart {p} (Similarity: {sim})')

In [ ]:
# ConceptNet Numberbatch embeddings
numberbatch_sim = 0.3
numberbatch = gensim.models.KeyedVectors.load_word2vec_format('./numberbatch-en.txt', binary=False)

# cosine similarity between ConceptNet Numberbatch embeddings
for f in foods:
    for p in parts:
        sim = numberbatch.similarity(f, p)
        if sim >= numberbatch_sim:
            print(f'[CN Numberbatch] {f} hasPart {p} (Similarity: {sim})')

In [ ]:
# NASARI embeddings 
# Sadly, the BabelNet synsets for core (bn:04772260n) does not exist in the NASARI embeddings and 
# for 'shell' we need to look for the concrete synset (bn:00071005n) instead 
parts_nasari = ['bn:00071005n', 'peel_(fruit)', 'plant_stem']
nasari_sim = 0.75
nasari = gensim.models.KeyedVectors.load_word2vec_format('./NASARI_embed_english.txt', binary=False)
               
# define function for finding the key based on the given concept name
def find_key(concept):
    concept_is_synset = "bn:" in concept
    keys = [key for key in nasari.index_to_key if concept in key.lower()]
    for key in keys:
        cut = key.split('__')[1].lower()
        if (cut == concept and not concept_is_synset) or (concept_is_synset and concept in key.lower()):
            return key
    return concept
    
# cosine similarity between NASARI embeddings
for f in foods:
    for p in parts_nasari:
        f_key = find_key(f)
        p_key = find_key(p)
        if (f_key in nasari.index_to_key) and (p_key in nasari.index_to_key):
            sim = nasari.similarity(f_key, p_key)
            if sim >= nasari_sim:
                print(f'[NASARI] {f} hasPart {p} (Similarity: {sim})')         

### Extract object properties using Recipe1M+ data

In [9]:
# read the (extracted) occurrence data from Recipe1M+
p_occurrences = "./part_occurrences.csv"
poc_dat = pd.read_csv(p_occurrences)
print(poc_dat)

          food  recipes   steps  bi_core_rec  bi_core_step  bi_shell_rec  \
0       almond    16897   16686            0             0             1   
1        apple    30101   27791           39            39             8   
2       banana    16927   11788            0             0             0   
3         bean    24254   16994            0             0             5   
4       cherry    12595   10407            0             0             0   
5      coconut    23622   31437            0             0            26   
6        lemon   111186  137495            0             0            15   
7         lime    37326   46503            0             0             9   
8        olive   115457  149795            0             0             0   
9       orange    38532   48280            0             0            14   
10      pepper   159174  181668            5             5             2   
11   pineapple    15178   19978           15            18            24   
12     pumpk

In [10]:
# set the thresholds for the 2-step ('two') and the bigram ('bi') methods
bi_recipe_thresh = 0.001
bi_step_thresh = 0.001
two_recipe_thresh = 0.01
two_step_thresh = 0.01

parts = ['core', 'shell', 'peel', 'stem']

# print object-part-relations
for idx, row in poc_dat.iterrows():
    for p in parts:
        bi_rec_ratio = row[f'bi_{p}_rec'] / row['recipes']
        bi_step_ratio = row[f'bi_{p}_step'] / row['steps']
        two_rec_ratio = row[f'two_{p}_rec'] / row['recipes']
        two_step_ratio = row[f'two_{p}_step'] / row['steps']
        
        if bi_rec_ratio >= bi_recipe_thresh and bi_step_ratio >= bi_step_thresh:
            fruit = row['food']
            rec_perc = '{:.2f}'.format(bi_rec_ratio*100)
            step_perc = '{:.2f}'.format(bi_step_ratio*100)
            print(f'[Bigram] {fruit} hasPart {p} ({rec_perc}% of Recipes & {step_perc}% of Steps)')
        
        if two_rec_ratio >= two_recipe_thresh and two_step_ratio >= two_step_thresh:
            fruit = row['food']
            rec_perc = '{:.2f}'.format(two_rec_ratio*100)
            step_perc = '{:.2f}'.format(two_step_ratio*100)
            print(f'[2-Step] {fruit} hasPart {p} ({rec_perc}% of Recipes & {step_perc}% of Steps)')

[Bigram] apple hasPart core (0.13% of Recipes & 0.14% of Steps)
[2-Step] apple hasPart core (3.61% of Recipes & 4.03% of Steps)
[Bigram] apple hasPart peel (0.18% of Recipes & 0.22% of Steps)
[2-Step] apple hasPart peel (3.31% of Recipes & 3.66% of Steps)
[Bigram] banana hasPart peel (0.15% of Recipes & 0.20% of Steps)
[2-Step] banana hasPart peel (2.84% of Recipes & 4.37% of Steps)
[2-Step] cherry hasPart stem (1.26% of Recipes & 1.63% of Steps)
[Bigram] lemon hasPart peel (4.54% of Recipes & 4.07% of Steps)
[2-Step] lemon hasPart peel (5.72% of Recipes & 5.17% of Steps)
[Bigram] lime hasPart peel (2.06% of Recipes & 1.85% of Steps)
[2-Step] lime hasPart peel (2.78% of Recipes & 2.51% of Steps)
[Bigram] orange hasPart peel (10.76% of Recipes & 9.78% of Steps)
[2-Step] orange hasPart peel (13.45% of Recipes & 12.44% of Steps)
[2-Step] pineapple hasPart core (1.94% of Recipes & 1.52% of Steps)
[Bigram] pineapple hasPart shell (0.16% of Recipes & 0.15% of Steps)
[2-Step] pineapple hasPar